# 🚀 Day 2 실습 1: Naive RAG 베이스라인 구축

## 학습 목표
- **기본 RAG 파이프라인** 구현 및 이해
- **벡터 검색**의 기본 원리 체험
- **성능 측정 시스템** 구축으로 개선점 파악
- **베이스라인 성능** 기록으로 향후 비교 기준 마련

### 💡 Naive RAG의 핵심
**"가장 단순하지만 동작하는" RAG 시스템**을 만드는 것이 목표입니다.

복잡한 기법 없이도 문서 검색-답변 생성이 어떻게 이루어지는지 직접 체험해보세요!

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# Day 2 실습 1: Naive RAG를 위한 핵심 라이브러리 설치
print("🚀 Day 2 실습 1: Naive RAG 베이스라인 구축")
print("📦 필요한 라이브러리 설치 중...")

# 벡터 데이터베이스 및 검색을 위한 라이브러리
!pip install -q faiss-cpu  # FAISS: Facebook AI Similarity Search (벡터 유사도 검색)
!pip install -q langchain langchain-community langchain-openai  # LangChain RAG 프레임워크
!pip install -q sentence-transformers  # 문장 임베딩 모델

# 데이터 처리 및 시각화
!pip install -q pandas numpy matplotlib seaborn plotly
!pip install -q tqdm  # 진행 상황 표시

# 한국어 자연어처리 (토큰화 등)
!pip install -q konlpy

# 문서 로딩 및 처리
!pip install -q beautifulsoup4 requests

print("✅ 라이브러리 설치 완료!")
print("💡 이제 Naive RAG의 핵심 구성요소를 단계별로 구축해보겠습니다.")

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
import time
import warnings
from tqdm import tqdm

# LangChain 핵심 컴포넌트
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.schema import Document

# Sentence Transformers (임베딩 모델)
from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib) - RAG 성능 분석 차트에서 한글이 깨지지 않도록 설정
# 베이스라인 성능 차트, 응답 시간 분석 등에서 한글 표시를 위해 필요
print("🔧 한글 폰트 설정 중...")
!apt-get update -qq
!apt-get install fonts-nanum -qq > /dev/null

import matplotlib.font_manager as fm

# 나눔바른고딕 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# 폰트 매니저에 폰트 추가 - RAG 성능 분석 그래프에서 한글 표시를 위해 필요
fm.fontManager.addfont(fontpath)

# matplotlib 설정 업데이트 - 모든 성능 분석 차트에서 한글이 정상적으로 표시됨
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',  # 기본 폰트를 나눔바른고딕으로 설정
    'axes.unicode_minus': False         # 음수 기호 표시 문제 해결
})

# 시각화 스타일 설정 - 깔끔한 RAG 성능 분석 차트를 위한 설정
plt.style.use('default')
sns.set_palette("husl")  # 구별하기 쉬운 색상 팔레트

print("✅ 한글 폰트 설정 완료 - RAG 성능 차트에서 한글이 정상 표시됩니다")
print("📦 라이브러리 import 완료!")

## 2. 샘플 데이터셋 준비

### 📚 실습용 한국어 문서 컬렉션
다양한 주제의 한국어 문서를 준비하여 Naive RAG의 검색 성능을 테스트합니다.
- AI/기술 관련 문서
- 정책/제도 관련 문서  
- 일반 상식 문서
- 시간적 정보가 포함된 문서

In [ ]:
def create_sample_documents() -> List[Dict[str, Any]]:
    """
    RAG 성능 테스트를 위한 다양한 한국어 샘플 문서 생성
    
    다양한 도메인과 난이도의 문서를 포함하여 Naive RAG의
    강점과 약점을 명확히 파악할 수 있도록 구성
    
    Returns:
        문서 리스트 (제목, 내용, 메타데이터 포함)
    """
    
    documents = [
        {
            "title": "ChatGPT와 생성형 AI의 등장",
            "content": """
            2022년 11월 OpenAI에서 공개한 ChatGPT는 대화형 인공지능 서비스로 큰 화제를 모았습니다. 
            GPT-3.5 기반으로 개발된 ChatGPT는 자연스러운 대화가 가능하며, 다양한 질문에 대해 
            상세하고 정확한 답변을 제공합니다. 이후 2023년 3월에는 더욱 발전된 GPT-4가 출시되어
            멀티모달 기능까지 지원하게 되었습니다. 생성형 AI 기술은 교육, 의료, 법률 등
            다양한 분야에서 활용되고 있으며, 특히 콘텐츠 생성과 코딩 지원 분야에서 혁신을 가져왔습니다.
            """,
            "metadata": {
                "category": "technology",
                "year": 2023,
                "source": "tech_news",
                "keywords": ["ChatGPT", "OpenAI", "GPT-4", "생성형AI", "인공지능"]
            }
        },
        {
            "title": "한국의 AI 국가전략과 정책 방향",
            "content": """
            대한민국 정부는 2024년 'AI 강국 코리아' 비전을 발표하며 인공지능 분야의 글로벌 리더십 
            확보를 위한 종합계획을 수립했습니다. 주요 정책으로는 AI 반도체 생태계 구축, 
            AI 데이터 댐 구축, K-클라우드 서비스 확산 등이 있습니다. 특히 정부는 2027년까지
            AI 분야에 총 9조 9천억원을 투자하기로 결정했습니다. 또한 AI 윤리 가이드라인을 마련하여
            안전하고 신뢰할 수 있는 AI 활용 환경을 조성하고 있습니다. 교육 분야에서는 AI 디지털 교과서
            도입을 통해 개인 맞춤형 학습을 지원할 계획입니다.
            """,
            "metadata": {
                "category": "policy",
                "year": 2024,
                "source": "government",
                "keywords": ["AI정책", "국가전략", "투자계획", "윤리가이드라인"]
            }
        },
        {
            "title": "머신러닝과 딥러닝의 차이점",
            "content": """
            머신러닝(Machine Learning)은 컴퓨터가 데이터를 통해 스스로 학습하는 인공지능의 한 분야입니다.
            전통적인 머신러닝 알고리즘으로는 선형회귀, 의사결정트리, SVM, 랜덤포레스트 등이 있습니다.
            딥러닝(Deep Learning)은 머신러닝의 하위 분야로, 인공신경망을 여러 층으로 쌓은 구조를 사용합니다.
            딥러닝은 이미지 인식, 음성 인식, 자연어 처리 등 복잡한 패턴 인식에서 뛰어난 성능을 보입니다.
            머신러닝은 상대적으로 적은 데이터로도 학습이 가능하지만, 딥러닝은 대량의 데이터와 
            높은 컴퓨팅 파워가 필요합니다. 최근에는 transformer 아키텍처 기반의 대규모 언어모델이 
            자연어 처리 분야에서 혁신을 가져오고 있습니다.
            """,
            "metadata": {
                "category": "education",
                "year": 2023,
                "source": "textbook",
                "keywords": ["머신러닝", "딥러닝", "신경망", "알고리즘", "ML", "DL"]
            }
        },
        {
            "title": "RAG(Retrieval-Augmented Generation) 기술 소개",
            "content": """
            RAG는 검색 증강 생성(Retrieval-Augmented Generation)의 줄임말로, 
            외부 지식베이스에서 관련 정보를 검색한 후 이를 바탕으로 답변을 생성하는 기술입니다.
            전통적인 언어모델은 학습 데이터에만 의존하지만, RAG는 실시간으로 최신 정보에 접근할 수 있습니다.
            RAG 시스템은 크게 검색기(Retriever)와 생성기(Generator)로 구성됩니다.
            검색기는 벡터 유사도를 기반으로 관련 문서를 찾고, 생성기는 검색된 맥락을 활용해 답변을 만듭니다.
            이 방식은 할루시네이션을 줄이고, 출처를 명확히 할 수 있다는 장점이 있습니다.
            최근에는 Naive RAG에서 Advanced RAG, Modular RAG로 발전하며 
            더욱 정교한 검색과 생성이 가능해졌습니다.
            """,
            "metadata": {
                "category": "technology",
                "year": 2024,
                "source": "research",
                "keywords": ["RAG", "검색증강생성", "retrieval", "벡터검색", "할루시네이션"]
            }
        },
        {
            "title": "파이썬 프로그래밍 기초",
            "content": """
            파이썬(Python)은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다.
            파이썬의 철학은 '간결하고 읽기 쉬운 코드'로, 'The Zen of Python'에 잘 나타나 있습니다.
            파이썬의 주요 특징으로는 인터프리터 언어, 동적 타이핑, 객체지향 프로그래밍 지원 등이 있습니다.
            데이터 사이언스 분야에서는 NumPy, Pandas, Matplotlib 등의 라이브러리가 핵심적으로 사용되며,
            웹 개발에서는 Django, Flask 같은 프레임워크가 인기입니다.
            머신러닝 분야에서는 scikit-learn, TensorFlow, PyTorch 등이 널리 활용됩니다.
            파이썬은 문법이 직관적이어서 초보자가 배우기 쉽고, 동시에 고급 기능도 풍부해서
            전문가들도 선호하는 언어입니다.
            """,
            "metadata": {
                "category": "programming",
                "year": 2023,
                "source": "tutorial",
                "keywords": ["Python", "파이썬", "프로그래밍", "라이브러리", "개발"]
            }
        },
        {
            "title": "벡터 데이터베이스와 임베딩",
            "content": """
            벡터 데이터베이스는 고차원 벡터 데이터를 효율적으로 저장하고 검색하는 데이터베이스입니다.
            기존 관계형 데이터베이스와 달리 벡터 간의 유사도를 기반으로 검색을 수행합니다.
            대표적인 벡터 데이터베이스로는 Pinecone, Weaviate, Chroma, FAISS 등이 있습니다.
            임베딩(Embedding)은 텍스트, 이미지, 오디오 등의 데이터를 고정된 크기의 벡터로 변환하는 과정입니다.
            텍스트 임베딩 모델로는 BERT, RoBERTa, Sentence-BERT, OpenAI의 text-embedding-ada-002 등이 있습니다.
            벡터 검색은 코사인 유사도, 유클리드 거리, 내적 등의 거리 함수를 사용하여 가장 유사한 벡터를 찾습니다.
            이 기술은 추천 시스템, 검색 엔진, 그리고 RAG 시스템에서 핵심적인 역할을 합니다.
            """,
            "metadata": {
                "category": "technology",
                "year": 2024,
                "source": "technical_guide",
                "keywords": ["벡터데이터베이스", "임베딩", "FAISS", "유사도검색", "embedding"]
            }
        },
        {
            "title": "2024년 AI 트렌드와 전망",
            "content": """
            2024년 인공지능 분야의 주요 트렌드는 멀티모달 AI, AGI 연구, AI 에이전트 등입니다.
            멀티모달 AI는 텍스트, 이미지, 음성 등 다양한 형태의 데이터를 동시에 처리할 수 있는 기술로,
            GPT-4V, Claude 3, Gemini 등의 모델에서 구현되고 있습니다.
            AI 에이전트는 자율적으로 작업을 수행하고 도구를 사용할 수 있는 시스템으로 주목받고 있습니다.
            오픈소스 모델의 발전도 눈에 띄는데, Llama 2, Mistral, Qwen 등이 상용 모델에 근접한 성능을 보입니다.
            AI 안전성과 정렬(Alignment) 연구도 중요한 화두로, Constitutional AI, RLHF 등의 기법이 발전하고 있습니다.
            기업들은 AI를 활용한 업무 자동화와 생산성 향상에 집중하고 있으며,
            특히 코드 생성, 문서 작성, 고객 서비스 등 분야에서 실용화가 가속화되고 있습니다.
            """,
            "metadata": {
                "category": "trend",
                "year": 2024,
                "source": "industry_report",
                "keywords": ["AI트렌드", "멀티모달", "AGI", "에이전트", "오픈소스"]
            }
        }
    ]
    
    return documents

# 샘플 문서 생성
sample_docs = create_sample_documents()

print(f"📚 샘플 문서 준비 완료: {len(sample_docs)}개 문서")
print("\n📋 문서 목록:")
for i, doc in enumerate(sample_docs, 1):
    print(f"  {i}. {doc['title']} ({doc['metadata']['category']}, {doc['metadata']['year']})")

print(f"\n💡 다양한 카테고리와 연도의 문서로 Naive RAG의 성능을 종합적으로 평가할 수 있습니다.")

## 3. 문서 전처리 및 청킹

### ✂️ 텍스트 분할 (Text Chunking)
긴 문서를 검색에 적합한 크기로 분할합니다.
- **Chunk 크기**: 너무 작으면 문맥 손실, 너무 크면 노이즈 증가
- **Overlap**: 청크 간 중복으로 문맥 연결성 유지
- **분할 기준**: 문장, 단락 등 의미 단위 고려

In [ ]:
def preprocess_documents(documents: List[Dict[str, Any]], 
                        chunk_size: int = 500, 
                        chunk_overlap: int = 50) -> List[Document]:
    """
    문서들을 RAG에 적합하게 전처리하고 청킹하는 함수
    
    Args:
        documents: 원본 문서 리스트
        chunk_size: 각 청크의 최대 글자 수
        chunk_overlap: 청크 간 중복 글자 수
        
    Returns:
        LangChain Document 객체 리스트
    """
    
    print(f"📝 문서 전처리 시작...")
    print(f"  청크 크기: {chunk_size} 글자")
    print(f"  청크 중복: {chunk_overlap} 글자")
    
    # 텍스트 분할기 설정
    # RecursiveCharacterTextSplitter: 문맥을 고려한 스마트한 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],  # 분할 우선순위
        length_function=len,
    )
    
    processed_docs = []
    chunk_stats = {"total_chunks": 0, "avg_chunk_size": 0, "chunks_per_doc": []}
    
    for doc_idx, doc in enumerate(tqdm(documents, desc="문서 청킹")):
        # 제목과 내용을 결합하여 더 풍부한 컨텍스트 제공
        full_text = f"제목: {doc['title']}\n\n{doc['content'].strip()}"
        
        # 문서를 청크로 분할
        chunks = text_splitter.split_text(full_text)
        
        # 각 청크를 LangChain Document로 변환
        for chunk_idx, chunk in enumerate(chunks):
            # 메타데이터에 추가 정보 포함
            metadata = doc['metadata'].copy()
            metadata.update({
                "doc_id": doc_idx,
                "doc_title": doc['title'],
                "chunk_id": chunk_idx,
                "chunk_size": len(chunk),
                "total_chunks_in_doc": len(chunks)
            })
            
            processed_docs.append(
                Document(
                    page_content=chunk,
                    metadata=metadata
                )
            )
        
        chunk_stats["chunks_per_doc"].append(len(chunks))
    
    # 통계 정보 계산
    chunk_stats["total_chunks"] = len(processed_docs)
    chunk_stats["avg_chunk_size"] = np.mean([len(doc.page_content) for doc in processed_docs])
    
    print(f"\n✅ 문서 전처리 완료!")
    print(f"  📊 전처리 결과:")
    print(f"    - 총 청크 수: {chunk_stats['total_chunks']}개")
    print(f"    - 평균 청크 크기: {chunk_stats['avg_chunk_size']:.1f} 글자")
    print(f"    - 문서당 청크 수: {np.mean(chunk_stats['chunks_per_doc']):.1f}개")
    print(f"    - 최대 청크 수: {max(chunk_stats['chunks_per_doc'])}개")
    
    return processed_docs, chunk_stats

# 문서 전처리 실행
processed_documents, chunking_stats = preprocess_documents(
    sample_docs,
    chunk_size=400,  # Naive RAG에서는 적당한 크기로 설정
    chunk_overlap=50  # 문맥 연결성을 위한 중복
)

# 샘플 청크 확인
print(f"\n📋 샘플 청크 미리보기:")
print(f"{'='*60}")
print(f"📄 첫 번째 청크:")
print(f"{processed_documents[0].page_content[:200]}...")
print(f"\n🏷️ 메타데이터:")
for key, value in list(processed_documents[0].metadata.items())[:5]:
    print(f"  {key}: {value}")
print(f"{'='*60}")

## 4. 임베딩 모델 설정 및 벡터스토어 구축

### 🧠 임베딩 모델 선택
한국어 문서에 적합한 임베딩 모델을 사용하여 벡터 표현을 생성합니다.
- **all-MiniLM-L6-v2**: 빠르고 효율적인 다국어 임베딩 모델
- **FAISS**: Facebook의 효율적인 벡터 유사도 검색 라이브러리

In [ ]:
def setup_vector_store(documents: List[Document], 
                      embedding_model_name: str = "BAAI/bge-m3") -> FAISS:
    """
    임베딩 모델을 설정하고 FAISS 벡터스토어를 구축하는 함수
    
    Args:
        documents: 전처리된 Document 리스트
        embedding_model_name: 사용할 임베딩 모델명
        
    Returns:
        FAISS 벡터스토어 객체
    """
    
    print(f"🧠 임베딩 모델 설정 중...")
    print(f"  모델: {embedding_model_name}")
    
    # HuggingFace 임베딩 모델 초기화
    # all-MiniLM-L6-v2: 384차원, 빠른 속도, 괜찮은 성능의 균형
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={'device': 'cpu'},  # GPU가 없는 환경에서도 실행 가능
        encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화로 코사인 유사도 최적화
    )
    
    print(f"✅ 임베딩 모델 로드 완료")
    
    # 벡터스토어 구축
    print(f"🔍 FAISS 벡터스토어 구축 중... (시간이 걸릴 수 있습니다)")
    
    start_time = time.time()
    
    # FAISS 벡터스토어 생성
    # 모든 문서를 임베딩하여 벡터 데이터베이스에 저장
    vectorstore = FAISS.from_documents(
        documents=documents,
        embedding=embeddings
    )
    
    build_time = time.time() - start_time
    
    print(f"✅ 벡터스토어 구축 완료!")
    print(f"  📊 구축 정보:")
    print(f"    - 인덱싱된 문서: {len(documents)}개")
    print(f"    - 임베딩 차원: {embeddings.client.get_sentence_embedding_dimension()}차원")
    print(f"    - 구축 시간: {build_time:.2f}초")
    print(f"    - 문서당 평균 시간: {build_time/len(documents):.3f}초")
    
    return vectorstore, embeddings

# 벡터스토어 구축
vector_store, embedding_model = setup_vector_store(processed_documents)

# 벡터스토어 저장 (재사용을 위해)
vector_store.save_local("naive_rag_vectorstore")
print(f"\n💾 벡터스토어 저장 완료: './naive_rag_vectorstore'")
print(f"💡 저장된 벡터스토어는 다음 실습에서 재사용할 수 있습니다.")

## 5. Naive RAG 파이프라인 구현

### 🔄 기본 RAG 워크플로우
1. **쿼리 임베딩**: 사용자 질문을 벡터로 변환
2. **유사도 검색**: 벡터스토어에서 Top-K 문서 검색
3. **컨텍스트 구성**: 검색된 문서들을 프롬프트에 포함
4. **답변 생성**: LLM을 통한 최종 답변 생성

In [ ]:
# Day 1 파인튜닝 모델을 사용하는 실제 LLM 클래스
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from typing import Optional, List, Any, Dict
from pydantic import Field

class Day1FinetunedLLM(LLM):
    """
    Day 1에서 파인튜닝된 모델을 로드하고 사용하는 실제 LLM 클래스
    여러분의 파인튜닝 모델 주소: https://huggingface.co/ryanu/my-exaone-raft-model
    """
    
    # Pydantic field declarations
    model_name: str = Field(default="ryanu/my-exaone-raft-model")
    base_model: str = Field(default="LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")
    usage_config: Dict[str, Any] = Field(default_factory=lambda: {
        "max_length": 512,
        "temperature": 0.7,
        "do_sample": True
    })
    tokenizer: Any = Field(default=None)
    model: Any = Field(default=None)
    model_loaded: bool = Field(default=False)
    
    class Config:
        arbitrary_types_allowed = True
    
    def __init__(self, model_name: str = "ryanu/my-exaone-raft-model", **kwargs):
        super().__init__(
            model_name=model_name,
            base_model="LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
            usage_config={
                "max_length": 512,
                "temperature": 0.7,
                "do_sample": True
            },
            **kwargs
        )
        
        print(f"🎯 Day 1 파인튜닝 모델 사용: {self.model_name}")
        print(f"📝 모델 주소: https://huggingface.co/{self.model_name}")
        
        self._load_model()
    
    def _load_model(self):
        """실제 파인튜닝된 모델 로드"""
        print(f"🔄 Day 1 파인튜닝 모델 로드 중: {self.model_name}")
        
        # Hugging Face에서 파인튜닝된 모델 직접 로드
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            trust_remote_code=True
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        
        # 패딩 토큰 설정
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
        print(f"✅ Day 1 파인튜닝 모델 로드 완료!")
        self.model_loaded = True
    
    @property
    def _llm_type(self) -> str:
        return "day1_finetuned_exaone_llm"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        
        # EXAONE 모델에 적합한 프롬프트 템플릿 사용
        formatted_prompt = f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다. 주어진 컨텍스트를 바탕으로 질문에 정확하고 유용한 답변을 제공해주세요.[|endofturn|]\n[|user|]{prompt}[|endofturn|]\n[|assistant|]"
        
        # 실제 파인튜닝된 모델로 추론
        inputs = self.tokenizer(
            formatted_prompt, 
            return_tensors="pt", 
            padding=True, 
            truncation=True,
            max_length=1024
        )
        
        # 모델과 같은 디바이스로 이동
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.usage_config.get("max_length", 512),
                temperature=self.usage_config.get("temperature", 0.7),
                do_sample=self.usage_config.get("do_sample", True),
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                num_return_sequences=1
            )
        
        # 응답 디코딩 (입력 프롬프트 제외)
        input_length = inputs["input_ids"].shape[1]
        response = self.tokenizer.decode(
            outputs[0][input_length:], 
            skip_special_tokens=True
        ).strip()
        
        return response if response else "죄송합니다. 적절한 답변을 생성할 수 없습니다."

def create_naive_rag_chain(vectorstore: FAISS, llm: LLM, k: int = 3) -> RetrievalQA:
    """
    Naive RAG 체인을 생성하는 함수
    
    Args:
        vectorstore: FAISS 벡터스토어
        llm: 언어 모델 (Day 1 파인튜닝 모델)
        k: 검색할 문서 개수 (Top-K)
        
    Returns:
        RetrievalQA 체인
    """
    
    print(f"🔗 Naive RAG 체인 생성 중...")
    print(f"  검색할 문서 수 (K): {k}")
    
    # 벡터스토어를 검색기(Retriever)로 변환
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k}
    )
    
    # RetrievalQA 체인 생성
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        verbose=False
    )
    
    print(f"✅ Naive RAG 체인 생성 완료!")
    
    return rag_chain

# Day 1 파인튜닝 모델 및 RAG 체인 생성
print("🚀 Day 1 파인튜닝 모델 기반 RAG 시스템 초기화 중...")
day1_llm = Day1FinetunedLLM("ryanu/my-exaone-raft-model")
naive_rag = create_naive_rag_chain(vector_store, day1_llm, k=3)

print(f"\n🎉 실제 파인튜닝 모델 기반 RAG 파이프라인 준비 완료!")

## 6. 성능 측정 시스템 구축

### 📊 RAG 성능 평가 메트릭
- **응답 시간**: 질문부터 답변까지 걸리는 시간
- **검색 정확도**: 관련 문서가 상위에 검색되는 비율
- **컨텍스트 길이**: 검색된 문서들의 총 길이
- **토큰 사용량**: LLM API 호출 시 사용되는 토큰 수
- **키워드 커버리지**: 답변이 질문의 핵심 키워드를 포함하는 정도

In [ ]:
class RAGPerformanceTracker:
    """
    RAG 시스템의 성능을 추적하고 분석하는 클래스
    
    다양한 메트릭을 자동으로 측정하고 시각화하여
    RAG 시스템의 강점과 약점을 파악할 수 있도록 돕습니다.
    """
    
    def __init__(self):
        self.performance_log = []
        self.query_count = 0
        
    def measure_query_performance(self, 
                                 rag_chain: RetrievalQA, 
                                 query: str, 
                                 expected_keywords: List[str] = None) -> Dict[str, Any]:
        """
        단일 쿼리의 성능을 측정하는 함수
        
        Args:
            rag_chain: RAG 체인
            query: 질문
            expected_keywords: 답변에 포함되어야 할 핵심 키워드
            
        Returns:
            성능 측정 결과
        """
        
        self.query_count += 1
        
        # 시작 시간 기록
        start_time = time.time()
        
        # RAG 체인 실행
        result = rag_chain({"query": query})
        
        # 종료 시간 기록
        end_time = time.time()
        response_time = end_time - start_time
        
        # 결과 추출
        answer = result["result"]
        source_docs = result["source_documents"]
        
        # 성능 메트릭 계산
        metrics = {
            "query_id": self.query_count,
            "query": query,
            "answer": answer,
            "response_time": response_time,
            "retrieved_docs_count": len(source_docs),
            "total_context_length": sum(len(doc.page_content) for doc in source_docs),
            "avg_context_length": np.mean([len(doc.page_content) for doc in source_docs]) if source_docs else 0,
            "unique_sources": len(set(doc.metadata.get("doc_title", "") for doc in source_docs)),
        }
        
        # 키워드 커버리지 계산
        if expected_keywords:
            answer_lower = answer.lower()
            covered_keywords = [kw for kw in expected_keywords if kw.lower() in answer_lower]
            metrics["keyword_coverage"] = len(covered_keywords) / len(expected_keywords)
            metrics["covered_keywords"] = covered_keywords
        else:
            metrics["keyword_coverage"] = None
        
        # 검색된 문서 정보
        doc_info = []
        for i, doc in enumerate(source_docs):
            doc_info.append({
                "rank": i + 1,
                "title": doc.metadata.get("doc_title", "Unknown"),
                "category": doc.metadata.get("category", "Unknown"),
                "chunk_size": len(doc.page_content),
                "content_preview": doc.page_content[:100] + "..."
            })
        
        metrics["retrieved_documents"] = doc_info
        metrics["timestamp"] = datetime.now().isoformat()
        
        # 로그에 추가
        self.performance_log.append(metrics)
        
        return metrics
    
    def get_performance_summary(self) -> Dict[str, Any]:
        """
        전체 성능 요약 통계 생성
        """
        
        if not self.performance_log:
            return {"message": "성능 로그가 없습니다. 먼저 쿼리를 실행해주세요."}
        
        response_times = [log["response_time"] for log in self.performance_log]
        context_lengths = [log["total_context_length"] for log in self.performance_log]
        keyword_coverages = [log["keyword_coverage"] for log in self.performance_log if log["keyword_coverage"] is not None]
        
        summary = {
            "total_queries": len(self.performance_log),
            "avg_response_time": np.mean(response_times),
            "min_response_time": np.min(response_times),
            "max_response_time": np.max(response_times),
            "avg_context_length": np.mean(context_lengths),
            "avg_keyword_coverage": np.mean(keyword_coverages) if keyword_coverages else None,
        }
        
        return summary
    
    def save_performance_log(self, filename: str = "naive_rag_performance.json"):
        """
        성능 로그를 JSON 파일로 저장
        """
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump({
                "logs": self.performance_log,
                "summary": self.get_performance_summary()
            }, f, ensure_ascii=False, indent=2, default=str)
        
        print(f"💾 성능 로그 저장 완료: {filename}")

# 성능 추적기 초기화
performance_tracker = RAGPerformanceTracker()

print("📊 RAG 성능 측정 시스템 준비 완료!")
print("💡 이제 다양한 질문으로 Naive RAG의 성능을 체계적으로 측정할 수 있습니다.")

## 7. Naive RAG 성능 테스트

### 🧪 다양한 질문 유형으로 테스트
Naive RAG의 강점과 약점을 파악하기 위해 여러 유형의 질문으로 테스트합니다.

In [ ]:
# 테스트 질문 세트 정의
test_queries = [
    {
        "query": "ChatGPT는 언제 출시되었나요?",
        "expected_keywords": ["2022년", "11월", "OpenAI", "출시"],
        "difficulty": "easy",
        "type": "factual"
    },
    {
        "query": "RAG 기술에 대해 설명해주세요",
        "expected_keywords": ["검색", "증강", "생성", "벡터", "Retrieval"],
        "difficulty": "medium",
        "type": "explanation"
    },
    {
        "query": "머신러닝과 딥러닝의 차이점은 무엇인가요?",
        "expected_keywords": ["머신러닝", "딥러닝", "신경망", "차이", "데이터"],
        "difficulty": "medium",
        "type": "comparison"
    },
    {
        "query": "파이썬으로 할 수 있는 일들을 알려주세요",
        "expected_keywords": ["파이썬", "데이터", "웹", "개발", "라이브러리"],
        "difficulty": "medium",
        "type": "application"
    },
    {
        "query": "2024년 AI 정책에 대한 투자 계획은?",
        "expected_keywords": ["2024", "정책", "투자", "9조", "AI강국"],
        "difficulty": "hard",
        "type": "specific_temporal"
    },
    {
        "query": "벡터 데이터베이스는 어떻게 작동하나요?",
        "expected_keywords": ["벡터", "데이터베이스", "유사도", "임베딩", "검색"],
        "difficulty": "hard",
        "type": "technical"
    }
]

def run_performance_test(rag_chain: RetrievalQA, 
                        tracker: RAGPerformanceTracker, 
                        test_queries: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    테스트 쿼리 세트로 RAG 성능을 종합적으로 평가
    
    Args:
        rag_chain: RAG 체인
        tracker: 성능 추적기
        test_queries: 테스트 쿼리 리스트
        
    Returns:
        측정 결과 리스트
    """
    
    print(f"🧪 Naive RAG 성능 테스트 시작!")
    print(f"📝 총 {len(test_queries)}개 질문으로 테스트...")
    print(f"{'='*80}")
    
    test_results = []
    
    for i, test_case in enumerate(tqdm(test_queries, desc="성능 테스트")):
        query = test_case["query"]
        expected_keywords = test_case["expected_keywords"]
        
        print(f"\n🔍 질문 {i+1}: {query}")
        
        # 성능 측정
        metrics = tracker.measure_query_performance(
            rag_chain, query, expected_keywords
        )
        
        # 추가 정보 저장
        metrics.update({
            "difficulty": test_case["difficulty"],
            "type": test_case["type"]
        })
        
        test_results.append(metrics)
        
        # 결과 미리보기
        print(f"  ⏱️ 응답시간: {metrics['response_time']:.3f}초")
        print(f"  📄 검색문서: {metrics['retrieved_docs_count']}개")
        print(f"  📏 컨텍스트: {metrics['total_context_length']}자")
        if metrics['keyword_coverage'] is not None:
            print(f"  🎯 키워드 커버리지: {metrics['keyword_coverage']:.1%}")
        print(f"  💬 답변: {metrics['answer'][:100]}...")
        
    print(f"\n{'='*80}")
    print(f"✅ 성능 테스트 완료!")
    
    return test_results

# 성능 테스트 실행
print("🚀 Naive RAG 종합 성능 테스트를 시작합니다!")
test_results = run_performance_test(naive_rag, performance_tracker, test_queries)

## 8. 성능 결과 분석 및 시각화

### 📊 Naive RAG의 강점과 약점 파악
측정된 성능 데이터를 시각화하여 개선이 필요한 영역을 식별합니다.

In [ ]:
def create_performance_analysis_dashboard(test_results: List[Dict[str, Any]], 
                                        tracker: RAGPerformanceTracker):
    import os
    import matplotlib as mpl
    import matplotlib.font_manager as fm

    font_candidates = [
        ("NanumBarunGothic", "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"),
        ("NanumGothic",      "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"),
    ]
    selected = None
    for name, path in font_candidates:
        if any(f.name == name for f in fm.fontManager.ttflist):
            selected = name; break
        if os.path.exists(path):
            fm.fontManager.addfont(path)
            try: fm._load_fontmanager(try_read_cache=False)  # mpl>=3.6
            except TypeError: fm._rebuild()                  # fallback
            if any(f.name == name for f in fm.fontManager.ttflist):
                selected = name; break
    if selected is None:
        selected = "DejaVu Sans"  # 최후 보루(한글 미포함일 수 있음)

    mpl.rcParams["font.family"] = selected
    mpl.rcParams["axes.unicode_minus"] = False
    """
    Naive RAG 성능 분석 대시보드 생성
    
    이 함수는 다양한 관점에서 RAG 성능을 시각화합니다:
    1. 📊 응답 시간 분석: 질문 유형별 응답 속도 비교
    2. 🎯 키워드 커버리지: 답변 정확도 측정  
    3. 📄 검색 품질: 검색된 문서 수와 컨텍스트 길이 분석
    4. 🔍 질문 난이도별 성능: 쉬운/중간/어려운 질문별 성능 차이
    """
    
    print("📊 Naive RAG 성능 분석 대시보드 생성 중...")
    
    # 데이터 준비
    df = pd.DataFrame(test_results)
    
    # 대시보드 생성 (2x2 서브플롯)
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🚀 Naive RAG 성능 분석 대시보드', fontsize=16, fontweight='bold')
    
    # 1. 응답 시간 분석 (좌상단)
    # 📊 의미: 질문 유형별로 응답 시간이 얼마나 다른지 분석
    # - 복잡한 질문일수록 검색과 생성에 더 오래 걸리는지 확인
    # - 일관된 응답 속도를 유지하는지 평가
    difficulties = df['difficulty'].tolist()
    response_times = df['response_time'].tolist()
    
    difficulty_order = ['easy', 'medium', 'hard']
    difficulty_colors = ['lightgreen', 'orange', 'lightcoral']
    
    for i, diff in enumerate(difficulty_order):
        diff_data = df[df['difficulty'] == diff]['response_time']
        if not diff_data.empty:
            axes[0, 0].bar(i, diff_data.mean(), 
                          color=difficulty_colors[i], alpha=0.7, 
                          label=f'{diff.capitalize()} (평균: {diff_data.mean():.3f}초)')
            
            # 개별 데이터 점 표시
            for j, value in enumerate(diff_data):
                axes[0, 0].scatter(i + (j-len(diff_data)/2)*0.1, value, 
                                  color='black', alpha=0.6, s=30)
    
    axes[0, 0].set_xlabel('질문 난이도')
    axes[0, 0].set_ylabel('응답 시간 (초)')
    axes[0, 0].set_title('⏱️ 난이도별 응답 시간 분석\n(점: 개별 질문, 막대: 평균)')
    axes[0, 0].set_xticks(range(len(difficulty_order)))
    axes[0, 0].set_xticklabels([d.capitalize() for d in difficulty_order])
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 키워드 커버리지 분석 (우상단)
    # 📊 의미: 답변이 질문의 핵심 키워드를 얼마나 잘 포함하는지 측정
    # - 높은 커버리지 = 관련성 높은 답변
    # - 낮은 커버리지 = 부정확하거나 불완전한 답변
    keyword_coverages = [r['keyword_coverage'] for r in test_results if r['keyword_coverage'] is not None]
    query_names = [f"Q{i+1}" for i in range(len(keyword_coverages))]
    
    bars = axes[0, 1].bar(query_names, keyword_coverages, 
                         color=['green' if cov >= 0.8 else 'orange' if cov >= 0.5 else 'red' 
                               for cov in keyword_coverages], alpha=0.7)
    
    # 커버리지 수치 표시
    for bar, cov in zip(bars, keyword_coverages):
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                        f'{cov:.1%}', ha='center', va='bottom', fontweight='bold')
    
    axes[0, 1].set_xlabel('질문')
    axes[0, 1].set_ylabel('키워드 커버리지')
    axes[0, 1].set_title('🎯 키워드 커버리지 분석\n(초록: 우수 ≥80%, 주황: 보통 ≥50%, 빨강: 부족 <50%)')
    axes[0, 1].set_ylim(0, 1.1)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 평균 커버리지 라인 추가
    avg_coverage = np.mean(keyword_coverages)
    axes[0, 1].axhline(y=avg_coverage, color='blue', linestyle='--', alpha=0.7,
                      label=f'평균: {avg_coverage:.1%}')
    axes[0, 1].legend()
    
    # 3. 검색 품질 분석 (좌하단)
    # 📊 의미: 검색된 문서의 품질과 다양성 분석
    # - 컨텍스트 길이: 너무 짧으면 정보 부족, 너무 길면 노이즈
    # - 고유 소스 수: 다양한 문서에서 정보를 가져왔는지 확인
    context_lengths = df['total_context_length'].tolist()
    unique_sources = df['unique_sources'].tolist()
    
    # 산점도로 컨텍스트 길이 vs 고유 소스 수 관계 시각화
    scatter = axes[1, 0].scatter(context_lengths, unique_sources, 
                                c=response_times, cmap='viridis', alpha=0.7, s=100)
    
    # 컬러바 추가 (응답 시간)
    cbar = plt.colorbar(scatter, ax=axes[1, 0])
    cbar.set_label('응답 시간 (초)', rotation=270, labelpad=15)
    
    # 각 점에 질문 번호 표시
    for i, (x, y) in enumerate(zip(context_lengths, unique_sources)):
        axes[1, 0].annotate(f'Q{i+1}', (x, y), xytext=(5, 5), 
                           textcoords='offset points', fontsize=9, alpha=0.8)
    
    axes[1, 0].set_xlabel('총 컨텍스트 길이 (문자 수)')
    axes[1, 0].set_ylabel('고유 문서 소스 수')
    axes[1, 0].set_title('📄 검색 품질 분석\n(색상: 응답시간, 위치: 컨텍스트 vs 다양성)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 질문 유형별 종합 성능 (우하단)
    # 📊 의미: 질문 유형(사실, 설명, 비교 등)별 종합 성능 비교
    # - 어떤 유형의 질문에 Naive RAG가 강한지/약한지 파악
    # - 향후 개선 방향 결정에 중요한 정보
    type_performance = df.groupby('type').agg({
        'response_time': 'mean',
        'keyword_coverage': 'mean',
        'total_context_length': 'mean'
    }).fillna(0)
    
    # 정규화 (0-1 스케일로 변환하여 비교 용이하게)
    normalized_perf = type_performance.copy()
    normalized_perf['response_time'] = 1 - (normalized_perf['response_time'] - normalized_perf['response_time'].min()) / (normalized_perf['response_time'].max() - normalized_perf['response_time'].min() + 1e-8)  # 응답시간은 낮을수록 좋음
    normalized_perf['keyword_coverage'] = normalized_perf['keyword_coverage']  # 이미 0-1 스케일
    normalized_perf['total_context_length'] = (normalized_perf['total_context_length'] - normalized_perf['total_context_length'].min()) / (normalized_perf['total_context_length'].max() - normalized_perf['total_context_length'].min() + 1e-8)
    
    # 히트맵으로 표시
    im = axes[1, 1].imshow(normalized_perf.T, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    
    # 축 라벨 설정
    axes[1, 1].set_xticks(range(len(normalized_perf.index)))
    axes[1, 1].set_xticklabels(normalized_perf.index, rotation=45, ha='right')
    axes[1, 1].set_yticks(range(len(normalized_perf.columns)))
    axes[1, 1].set_yticklabels(['응답속도', '키워드커버리지', '컨텍스트길이'])
    
    # 수치 표시
    for i in range(len(normalized_perf.index)):
        for j in range(len(normalized_perf.columns)):
            text = axes[1, 1].text(i, j, f'{normalized_perf.iloc[i, j]:.2f}',
                                  ha="center", va="center", color="black", fontweight='bold')
    
    axes[1, 1].set_title('🔍 질문 유형별 종합 성능\n(1.0: 최고, 0.0: 최저)')
    
    # 컬러바 추가
    cbar2 = plt.colorbar(im, ax=axes[1, 1])
    cbar2.set_label('정규화 성능 점수', rotation=270, labelpad=15)
    
    plt.tight_layout()
    
    # 그래프 저장
    plt.savefig('naive_rag_performance_dashboard.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()
    
    print("✅ 성능 분석 대시보드 생성 완료")
    print("💾 저장된 파일: naive_rag_performance_dashboard.png")
    
    # 성능 요약 출력
    summary = tracker.get_performance_summary()
    print(f"\n📋 Naive RAG 성능 요약:")
    print(f"  📊 총 쿼리: {summary['total_queries']}개")
    print(f"  ⏱️ 평균 응답시간: {summary['avg_response_time']:.3f}초")
    print(f"  📏 평균 컨텍스트 길이: {summary['avg_context_length']:.0f}자")
    if summary['avg_keyword_coverage']:
        print(f"  🎯 평균 키워드 커버리지: {summary['avg_keyword_coverage']:.1%}")
    
    return fig, summary

# 성능 분석 대시보드 생성
dashboard_fig, performance_summary = create_performance_analysis_dashboard(test_results, performance_tracker)

# 성능 로그 저장
performance_tracker.save_performance_log("naive_rag_baseline.log")

## 9. Naive RAG의 한계점 분석

### 🔍 발견된 주요 문제점들
측정 결과를 바탕으로 Naive RAG의 구조적 한계를 분석합니다.

In [ ]:
def analyze_naive_rag_limitations(test_results: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Naive RAG의 한계점을 체계적으로 분석하는 함수
    
    측정된 성능 데이터를 바탕으로 다음 단계 개선 방향을 제시합니다.
    
    Returns:
        한계점 분석 결과 및 개선 제안
    """
    
    print("🔍 Naive RAG 한계점 분석 중...")
    
    limitations = {
        "검색 품질 문제": [],
        "응답 품질 문제": [],
        "효율성 문제": [],
        "확장성 문제": []
    }
    
    improvements_needed = []
    
    # 1. 키워드 커버리지 분석
    keyword_coverages = [r['keyword_coverage'] for r in test_results if r['keyword_coverage'] is not None]
    low_coverage_count = sum(1 for cov in keyword_coverages if cov < 0.6)
    
    if low_coverage_count > len(keyword_coverages) * 0.3:  # 30% 이상이 낮은 커버리지
        limitations["응답 품질 문제"].append({
            "문제": "낮은 키워드 커버리지",
            "설명": f"{low_coverage_count}/{len(keyword_coverages)} 질문에서 60% 미만의 키워드 커버리지",
            "원인": "단순 벡터 유사도 검색의 한계, 동의어/유의어 처리 부족"
        })
        improvements_needed.append("Multi-Query RAG로 다양한 표현 검색")
        improvements_needed.append("Hybrid Search (BM25 + Vector) 도입")
    
    # 2. 응답 시간 일관성 분석
    response_times = [r['response_time'] for r in test_results]
    time_std = np.std(response_times)
    time_mean = np.mean(response_times)
    
    if time_std > time_mean * 0.5:  # 표준편차가 평균의 50% 이상
        limitations["효율성 문제"].append({
            "문제": "응답 시간 편차 큼",
            "설명": f"응답시간 편차: {time_std:.3f}초 (평균: {time_mean:.3f}초)",
            "원인": "질문 복잡도에 따른 처리 시간 차이, 캐싱 부재"
        })
        improvements_needed.append("응답 캐싱 시스템 구축")
        improvements_needed.append("동적 K값 조정으로 처리 시간 최적화")
    
    # 3. 컨텍스트 길이 분석
    context_lengths = [r['total_context_length'] for r in test_results]
    long_context_count = sum(1 for length in context_lengths if length > 2000)
    
    if long_context_count > 0:
        limitations["효율성 문제"].append({
            "문제": "과도한 컨텍스트 길이",
            "설명": f"{long_context_count}개 질문에서 2000자 이상의 컨텍스트",
            "원인": "고정된 Top-K 검색, 중복/관련없는 정보 포함"
        })
        improvements_needed.append("Re-ranking으로 관련도 높은 문서만 선별")
        improvements_needed.append("동적 컨텍스트 길이 조정")
    
    # 4. 문서 다양성 분석
    unique_sources = [r['unique_sources'] for r in test_results]
    low_diversity_count = sum(1 for sources in unique_sources if sources <= 1)
    
    if low_diversity_count > 0:
        limitations["검색 품질 문제"].append({
            "문제": "낮은 정보원 다양성",
            "설명": f"{low_diversity_count}개 질문에서 단일 문서에만 의존",
            "원인": "유사한 청크들이 상위에 랭크되는 현상, 다양성 고려 부족"
        })
        improvements_needed.append("Diversity Re-ranking 도입")
        improvements_needed.append("MMR (Maximal Marginal Relevance) 적용")
    
    # 5. 질문 유형별 성능 편차 분석
    type_coverages = {}
    for result in test_results:
        if result['keyword_coverage'] is not None:
            q_type = result['type']
            if q_type not in type_coverages:
                type_coverages[q_type] = []
            type_coverages[q_type].append(result['keyword_coverage'])
    
    type_avg_coverage = {k: np.mean(v) for k, v in type_coverages.items()}
    worst_type = min(type_avg_coverage, key=type_avg_coverage.get)
    best_type = max(type_avg_coverage, key=type_avg_coverage.get)
    
    if type_avg_coverage[best_type] - type_avg_coverage[worst_type] > 0.3:  # 30% 이상 차이
        limitations["검색 품질 문제"].append({
            "문제": "질문 유형별 성능 편차",
            "설명": f"{worst_type}({type_avg_coverage[worst_type]:.1%}) vs {best_type}({type_avg_coverage[best_type]:.1%})",
            "원인": "획일적 검색 전략, 질문 유형 특성 미고려"
        })
        improvements_needed.append("질문 유형별 라우팅 시스템")
        improvements_needed.append("Modular RAG 아키텍처 도입")
    
    # 6. 확장성 문제 (구조적 한계)
    limitations["확장성 문제"].extend([
        {
            "문제": "메타데이터 활용 부족",
            "설명": "문서의 연도, 카테고리, 신뢰도 등 메타정보를 검색에 활용하지 않음",
            "원인": "순수 벡터 유사도에만 의존하는 단순한 구조"
        },
        {
            "문제": "실시간 정보 업데이트 어려움",
            "설명": "새로운 문서 추가 시 전체 인덱스 재구축 필요",
            "원인": "정적 벡터스토어 구조"
        },
        {
            "문제": "복합 질문 처리 한계",
            "설명": "여러 주제가 섞인 질문이나 추론이 필요한 질문에 약함",
            "원인": "단순한 검색-생성 파이프라인"
        }
    ])
    
    # 개선 제안 우선순위 정리
    improvements_needed.extend([
        "Self-Query Retriever로 메타데이터 필터링",
        "Query Decomposition으로 복합 질문 처리",
        "실시간 인덱스 업데이트 시스템"
    ])
    
    analysis_result = {
        "limitations": limitations,
        "improvements_needed": list(set(improvements_needed)),  # 중복 제거
        "next_steps": [
            "02_naive_failure_analysis.ipynb: 실패 케이스 상세 분석",
            "03_advanced_query_refinement.ipynb: Query Refinement 기법 적용",
            "04_metadata_filtering.ipynb: 메타데이터 기반 필터링",
            "05_hybrid_search_rerank.ipynb: 하이브리드 검색 및 리랭킹"
        ]
    }
    
    return analysis_result

# 한계점 분석 실행
limitation_analysis = analyze_naive_rag_limitations(test_results)

print("\n📋 Naive RAG 한계점 분석 결과:")
print("="*80)

for category, issues in limitation_analysis["limitations"].items():
    if issues:  # 문제가 있는 카테고리만 출력
        print(f"\n🔴 {category}:")
        for issue in issues:
            print(f"  • {issue['문제']}: {issue['설명']}")
            print(f"    원인: {issue['원인']}")

print(f"\n🛠️ 개선이 필요한 영역:")
for i, improvement in enumerate(limitation_analysis["improvements_needed"], 1):
    print(f"  {i}. {improvement}")

print(f"\n🚀 다음 실습 단계:")
for step in limitation_analysis["next_steps"]:
    print(f"  📝 {step}")

# 분석 결과 저장
with open("naive_rag_limitations_analysis.json", 'w', encoding='utf-8') as f:
    json.dump(limitation_analysis, f, ensure_ascii=False, indent=2, default=str)

print(f"\n💾 한계점 분석 결과 저장: naive_rag_limitations_analysis.json")

## 10. 📋 Naive RAG 베이스라인 요약

### ✅ 완료된 작업
1. **환경 설정**: FAISS, LangChain, 임베딩 모델 준비
2. **데이터 준비**: 다양한 도메인의 한국어 문서 7개
3. **문서 전처리**: 400자 청크, 50자 오버랩으로 분할
4. **벡터스토어 구축**: all-MiniLM-L6-v2 임베딩으로 FAISS 인덱스 생성
5. **RAG 파이프라인**: 기본 검색-생성 체인 구현
6. **성능 측정**: 6가지 질문 유형으로 종합 평가
7. **결과 분석**: 강점/약점 파악 및 개선 방향 도출

### 📊 측정된 베이스라인 성능
- **평균 응답시간**: 검색+생성 통합 시간 측정
- **키워드 커버리지**: 답변의 정확도 대리 지표
- **검색 품질**: 컨텍스트 길이, 문서 다양성
- **질문 유형별 성능**: 사실형, 설명형, 비교형 등

### 🔍 발견된 주요 한계점
- **검색 품질**: 동의어 처리 부족, 문서 다양성 제한
- **응답 품질**: 키워드 커버리지 편차, 질문 유형별 성능 차이
- **효율성**: 응답 시간 일관성 부족, 과도한 컨텍스트 길이
- **확장성**: 메타데이터 미활용, 복합 질문 처리 한계

### 🚀 다음 단계 Preview
다음 실습에서는 이러한 한계점들을 하나씩 해결해나갑니다:
- **실패 케이스 분석**: 구체적인 문제 상황 재현
- **Advanced RAG**: Query Refinement, Hybrid Search, Re-ranking
- **Modular RAG**: 지능형 라우팅, 조건부 체이닝

### 💡 핵심 인사이트
Naive RAG는 **빠른 구현과 명확한 구조**라는 장점이 있지만, 
**실제 프로덕션 환경**에서는 다양한 개선 기법이 필요합니다.

이 베이스라인이 향후 모든 개선 기법의 **성능 비교 기준**이 됩니다!

In [ ]:
# 최종 요약 및 마무리
print("🎉 Day 2 실습 1: Naive RAG 베이스라인 구축 완료!")
print("=" * 80)

# 생성된 파일들 요약
generated_files = [
    "naive_rag_vectorstore/ (FAISS 벡터스토어)",
    "naive_rag_baseline.log (성능 측정 로그)",
    "naive_rag_performance_dashboard.png (성능 분석 차트)",
    "naive_rag_limitations_analysis.json (한계점 분석 결과)"
]

print(f"📁 생성된 파일들:")
for file in generated_files:
    print(f"  📄 {file}")

# 주요 성능 지표 요약 출력
if performance_summary:
    print(f"\n📊 Naive RAG 베이스라인 성능:")
    print(f"  ⏱️ 평균 응답시간: {performance_summary['avg_response_time']:.3f}초")
    print(f"  📏 평균 컨텍스트: {performance_summary['avg_context_length']:.0f}자")
    if performance_summary['avg_keyword_coverage']:
        print(f"  🎯 평균 키워드 커버리지: {performance_summary['avg_keyword_coverage']:.1%}")
    print(f"  📝 총 테스트 쿼리: {performance_summary['total_queries']}개")

print(f"\n🔄 다음 실습:")
print(f"  📝 02_naive_failure_analysis.ipynb")
print(f"     → Naive RAG가 실패하는 구체적인 케이스들을 분석하고")
print(f"     → 실시간 성능 모니터링 대시보드를 구축합니다")

print(f"\n💡 베이스라인 확립 완료!")
print(f"🚀 이제 Advanced RAG 기법들로 단계적 개선을 시작할 수 있습니다!")

print(f"\n" + "=" * 80)
print(f"🎯 Naive RAG → Advanced RAG → Modular RAG 여정의 첫 걸음 완성! 🎯")